In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
%pylab inline

In [ ]:
ulist = pd.read_csv('data/user_list.csv')
ulist.info()

In [ ]:
cpltr = pd.read_csv('data/coupon_list_train.csv')
cpltr.info()

In [ ]:
cpdtr = pd.read_csv('data/coupon_detail_train.csv')
cpdtr.info()

In [ ]:
cplte = pd.read_csv('data/coupon_list_test.csv')
cplte.info()

In [ ]:
train = pd.merge(cpdtr, cpltr, how='inner', on='COUPON_ID_hash')
train = train[["COUPON_ID_hash","USER_ID_hash",
    "GENRE_NAME","DISCOUNT_PRICE","PRICE_RATE",
    "USABLE_DATE_MON","USABLE_DATE_TUE","USABLE_DATE_WED","USABLE_DATE_THU",
    "USABLE_DATE_FRI","USABLE_DATE_SAT","USABLE_DATE_SUN","USABLE_DATE_HOLIDAY",
    "USABLE_DATE_BEFORE_HOLIDAY","large_area_name","ken_name","small_area_name"]]
train.info()

In [ ]:
cplte['USER_ID_hash'] = 'dummyuser'
cpchar = cplte[["COUPON_ID_hash","USER_ID_hash",
    "GENRE_NAME","DISCOUNT_PRICE","PRICE_RATE",
    "USABLE_DATE_MON","USABLE_DATE_TUE","USABLE_DATE_WED","USABLE_DATE_THU",
    "USABLE_DATE_FRI","USABLE_DATE_SAT","USABLE_DATE_SUN","USABLE_DATE_HOLIDAY",
    "USABLE_DATE_BEFORE_HOLIDAY","large_area_name","ken_name","small_area_name"]]
cpchar[:3]

In [ ]:
cpchar.info()

In [ ]:
train = pd.concat([train, cpchar])
train.index = pd.Index(range(len(train)))
train.info()

In [ ]:
train = train.fillna(1)

In [ ]:
from math import log10
from math import pow
import sys

In [ ]:
train.loc[train.DISCOUNT_PRICE <= 0, 'DISCOUNT_PRICE'] = 0.00001
train.DISCOUNT_PRICE = train.DISCOUNT_PRICE.apply(lambda x: 1. / log10(x))
train.PRICE_RATE = train.PRICE_RATE.apply(lambda x: pow(x, 2) / 10000.)

In [ ]:
train[:3]

In [ ]:
train = pd.get_dummies(train, columns=['GENRE_NAME', 'large_area_name', 'ken_name', 'small_area_name'])

In [ ]:
test = train.loc[train.USER_ID_hash == 'dummyuser']
test.info()

In [ ]:
test.columns

In [ ]:
# test <- test[,-2]
del test['USER_ID_hash']
test.info()

In [ ]:
train = train.loc[train.USER_ID_hash != 'dummyuser']
train.info()

In [ ]:
train.columns

In [ ]:
del train['COUPON_ID_hash']
grouped = train.groupby(['USER_ID_hash'])
uchar = grouped.mean()
uchar.DISCOUNT_PRICE = 1
uchar.PRICE_RATE = 1

In [ ]:
W = np.diag(([2.05] * 13) + ([2.01] * 1) + ([-0.13] * 1) + ([0] * 9) + ([0.51] * 9) + ([1.01] * 47) + ([4.75] * 55))
W.shape

In [ ]:
test_mx = test.copy()
del test_mx['COUPON_ID_hash']
r1 = np.dot((uchar.loc[:,:]).as_matrix(), W)
r2 = np.dot(r1, test_mx.as_matrix().T)
r2.shape